<h1>Bank Customer Churn Prediction</h1>

In [ ]:
#Autor: Willy Ferreira
#Data: 28/02/2023

<h3>1. Instalação e importação dos pacotes</h3>

<h4>1.1. Instalação</h4>

In [ ]:
##Processamento e análise de dados
%pip install numpy

##Biblioteca para manipulação de dados
%pip install pandas

##Criação de gráficos interativos 
%pip install plotly

##Análise rápida de dados
%pip install pandas_profiling

##Interface gráfica para Jupyter
#%pip install ipywidgets

##Criação de gráficos
%pip install matplotlib

#Criação de dashboards interativos
#%pip install dash

<h4>1.2. Importação</h4>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
#import dash
#from dash import Dash, html, dcc
# 
#from pandas_profiling import ProfileReport
#import ipywidgets
import matplotlib.pyplot

Leitura do arquivo CSV e demais tratamentos

In [ ]:
#Leitura do arquivo CSV
df_original = pd.read_csv(filepath_or_buffer = "willyferreira/bank/Bank Customer Churn Prediction.csv", sep = ",", header = 0, index_col = False)

#Alteração do nome das colunas
df = df_original.rename(columns = 
            {"customer_id": "id_cliente",
             "credit_score": "score_de_credito",
             "country": "pais",
             "gender": "genero",
             "age": "idade",
             "tenure": "idade_abertura_cc",
             "balance": "saldo_cc",
             "products_number": "qtd_produtos",
             "credit_card": "cartao_de_credito",
             "active_member": "ativo",
             "estimated_salary": "salario_estimado"})

In [ ]:
#Mostrar o data frame
display(df)

In [ ]:
#Informações sobre o data frame
df.info()

Análise Exploratória - Score de Crédito

In [ ]:
df.score_de_credito.describe()

In [ ]:
app = Dash(__name__)

fig = px.box(data_frame = df, x = "score_de_credito")

app.layout = html.Div(children = [
        html.H1(children = "Score de Crédito"),
    
dcc.Graph(id = "example-graph", figure = fig)
]
)
if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
#Gráfico - Score de Crédito
fig_score = px.box(
    data_frame = df, 
    x = "score_de_credito",
    title = "Score de Crédito"
    #,
    #notched = False,
    #hover_data = ["churn"]
    )
fig_score

In [ ]:
df_pais = df["pais"].count()
df_pais

In [ ]:
df_pais

In [ ]:
#Gráfico - Clientes por país
fig_pais = px.pie(
    data_frame = df, 
    names = "pais",
    title = "Clientes por País",
    color = "pais",
    color_discrete_map = {"France": "cyan",
                          "Germany": "royalblue",
                          "Spain": "darkblue"},
    hole = .4
                )
fig_pais

In [ ]:
df.idade.describe()

In [ ]:
#Gráfico - Idade
fig_idade = px.box(
    data_frame = df, 
    x = "idade",
    title = "Idade"
    #,
    #notched = False,
    #hover_data = ["churn"]
    )
fig_idade

In [ ]:
df.idade_abertura_cc.describe()

In [ ]:
#Gráfico - Tempo de abertura da conta
fig_idade_abertura_cc= px.box(
    data_frame = df, 
    x = "idade_abertura_cc",
    title = "Tempo de abertura da conta"
    )
fig_idade_abertura_cc

In [ ]:
df.saldo_cc.describe()

In [ ]:
#Gráfico - Saldo em Conta Corrente
fig_saldo_cc= px.box(
    data_frame = df, 
    x = "saldo_cc",
    title = "Saldo em Conta Corrente"
    )
fig_saldo_cc

In [ ]:
df.qtd_produtos.describe()

In [ ]:
df.qtd_produtos.value_counts()

In [ ]:
#Gráfico - Quantidade de Produtos
fig_qtd_produtos= px.box(
    data_frame = df, 
    x = "qtd_produtos",
    title = "Quantidade de Produtos"
    )
fig_qtd_produtos

In [ ]:
#Gráfico - Clientes por quantidade de produtos
fig_qtd_produtos_pie = px.pie(
    data_frame = df, 
    names = "qtd_produtos",
    title = "Clientes por quantidade de produtos",
    color = "qtd_produtos",
    color_discrete_map = {1: "cyan",
                          2: "royalblue",
                          3: "darkblue",
                          4: "lightblue"},
    hole = .5
                )
fig_qtd_produtos_pie

Correlações

In [ ]:
df_corr = df.corr(method = "pearson")

In [ ]:
import plotly.graph_objects as go

fig_corr = go.Figure(
    data = go.Heatmap(
        z = df_corr,
        x = df_corr.columns,
        y = df_corr.columns
    )
)

fig_corr.show()

In [ ]:
fig_corr_arred = px.imshow(
    round(df_corr, 2),
    text_auto = True,
    aspect = "auto"
)

fig_corr_arred.show()

In [ ]:
#profile = ProfileReport(df_bank)
#profile.to_widgets()
#profile.to_file("bank_churn.html")
#profile.to_notebook_iframe()

In [ ]:
df["pais"].unique()
#df_bank["pais"].nunique()

In [ ]:
#ver as colunas do data frame
df.columns
#ver a quantidade de linhas
df.index
#mostrar estatísticas rápidas do data frame
df.describe()
#pivotar o data frame
df.T
#monstrando o data frame ordenado pelas colunas crescentes
df.sort_index(axis = 1, ascending = True)
#mostrando o data frame ordenado por uma coluna
df.sort_values(by = "churn")
#mostrando 5 primeiras linhas e 2 colunas específicas
df.loc[0:5:, ["ativo", "idade"]]
#mostrar 3 primeiras linhas e todas as colunas
df.iloc[0:3, :]
#mostrar uma posição específica
df.iloc[0]

